In [1]:
import random
import time
from copy import deepcopy
import os
import numpy as np
import collections
import matplotlib.pyplot as plt
import scikits.bootstrap as bootstrap
import warnings
warnings.filterwarnings('ignore') # Danger, Will Robinson! (not a scalable hack, and may surpress other helpful warning other than for ill-conditioned bootstrapped CI distributions)
import testcopy as test


In [6]:
# test.run()
random_image_paths = test.create_random_images()
trainloader = test.load_random_images(random_image_paths)
activations = test.get_activations(trainloader)
for k, v in activations.items():
    print(np.array(v).shape)

torch.Size([32, 3, 3, 3])
torch.Size([64, 32, 3, 3])
torch.Size([128, 64, 3, 3])
(50, 4, 32, 30, 30)
(50, 4, 64, 28, 28)
(50, 4, 128, 26, 26)


In [3]:
def evolution(generations, population_size, num_children, tournament_size, num_winners=1, evolution_type="regularized"):
    """Algorithm for regularized evolution (i.e. aging evolution).

    Follows "Algorithm 1" in Real et al. "Regularized Evolution for Image
    Classifier Architecture Search".

    Args:
      generations: the number of generations the algorithm should run for.
      population_size: the number of individuals to keep in the population.
      tournament_size: the number of individuals that should participate in each
          tournament.

    Returns:
      history: a list of `Model` instances, representing all the models computed
          during the evolution experiment.
    """
    population = collections.deque()
    history = []  # Not used by the algorithm, only used to report results.
    solutions_over_time = []
    fitness_over_time = []
    novelty_over_time = []

    # Initialize the population with random models.
    while len(population) < population_size:
        activations = test.get_activations(trainloader).values()[2]
        model.accuracy = train_and_eval(model.arch)
        population.append(model)
        history.append(model)
    
    for ind in population:
        ind.novelty = compute_novelty(ind.arch, history)
        
    # Carry out evolution in cycles. Each cycle produces a model and removes
    # another.
    for i in range(generations):
#     while len(history) < generations:
        
        parents = []
#         if evolution_type == "regularized":
#             while len(sample) < sample_size:
#             # Inefficient, but written this way for clarity. In the case of neural
#             # nets, the efficiency of this line is irrelevant because training neural
#             # nets is the rate-determining step.
#                 candidate = random.choice(list(population))
#                 sample.append(candidate)

#             # The parent is the best model in the sample.
#             parent = max(sample, key=lambda i: i.accuracy)
#             parents.append(parent)
#         else:    
        while len(parents) < num_children and evolution_type != "random":
        # Sample randomly chosen models from the current population.
            tournament = []
            while len(tournament) < tournament_size:
            # Inefficient, but written this way for clarity. In the case of neural
            # nets, the efficiency of this line is irrelevant because training neural
            # nets is the rate-determining step.
                candidate = random.choice(list(population))
                tournament.append(candidate)

            # The parent is the best model in the sample.
            if evolution_type == "novelty":
                parents.extend(sorted(tournament, key=lambda i: i.novelty, reverse=True)[:num_winners])
            elif evolution_type == "mixed":
                novelties = collections.deque(sorted(tournament, key=lambda i: i.novelty, reverse=True))
                fitnesses = collections.deque(sorted(tournament, key=lambda i: i.accuracy, reverse=True))
                for i in range(num_winners):
                    if random.uniform(0, 1) <= .5:
                        parents.append(novelties.popleft())
                    else:
                        parents.append(fitnesses.popleft())
            else:
                parents.extend(sorted(tournament, key=lambda i: i.accuracy, reverse=True)[:num_winners])
        

        # Create the child model and store it.
        for parent in parents:
            child = Model()
            child.arch = mutate_arch(parent.arch)
            child.accuracy = train_and_eval(child.arch)
            population.append(child)
            history.append(child)
            if len(history) > population_size + num_children:
                history=history[1:]
            population[-1].novelty = compute_novelty(child.arch, history)


            # Remove the oldest model.
        if evolution_type == 'regularized':
            population = list(population)[:population_size]
            
        if evolution_type == 'fitness':
            population = sorted(population, key=lambda i: i.accuracy, reverse=True)[:population_size]
                
        if evolution_type == 'novelty':
            population = sorted(population, key=lambda i: i.novelty, reverse=True)[:population_size]
            
        if evolution_type == 'mixed':
            population = sorted(population, key=lambda i: i.accuracy, reverse=True)
            new_population = population[:int(.5*population_size)]
            population = new_population + sorted(population, key=lambda i: i.novelty, reverse=True)[:int(population_size*.5)]
            population = sorted(population, key=lambda i: i.accuracy, reverse=True)[:population_size]
        
        if evolution_type == 'random':
            population = [sorted(population, key=lambda i: i.accuracy, reverse=True)[0]]
            history = []
            while len(population) < population_size:
                model = Model()
                model.arch = random_architecture()
                model.accuracy = train_and_eval(model.arch)
                population.append(model)
                history.append(model)
    
            for ind in population:
                ind.novelty = compute_novelty(ind.arch, history)
        
#         print([x.accuracy for x in sorted(population, key=lambda i: i.accuracy, reverse=True)])
        fitness_over_time.append(1+(sorted(population, key=lambda i: i.accuracy, reverse=True)[0].accuracy))
        solutions_over_time.append((sorted(population, key=lambda i: i.accuracy, reverse=True)[0].arch))
        novelty_over_time.append(np.mean([x.novelty for x in population]))

    return history, solutions_over_time, np.array(fitness_over_time), novelty_over_time

Files already downloaded and verified
Files already downloaded and verified


(Dataset CIFAR10
     Number of datapoints: 50000
     Root location: data
     Split: Train
     StandardTransform
 Transform: Compose(
                ToTensor()
                Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
            ),
 Dataset CIFAR10
     Number of datapoints: 10000
     Root location: data
     Split: Test
     StandardTransform
 Transform: Compose(
                ToTensor()
                Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
            ),
 ('plane',
  'car',
  'bird',
  'cat',
  'deer',
  'dog',
  'frog',
  'horse',
  'ship',
  'truck'))

AttributeError: 'function' object has no attribute 'load_CIFAR_10'